# Classification Model - XGBoost

In [583]:
#Importing all libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import math
import os
from itertools import combinations
from scipy.stats import chi2_contingency
import category_encoders as ce
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.model_selection import GridSearchCV
from xgboost import XGBClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report, make_scorer
from sklearn.utils.class_weight import compute_sample_weight
from imblearn.over_sampling import RandomOverSampler
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline
from catboost import CatBoostClassifier
from sklearn.utils.class_weight import compute_class_weight
from sklearn.model_selection import RandomizedSearchCV

In [584]:
print(os.getcwd())

c:\Users\ntama\Documentos\Data Science\Codecademy\projects\Date-A-Scientist - Final Project\OKCupid-Date-A-Scientist-Starter\date-a-scientist-project\notebooks


## Importing predictors and target - train, test and validation subsets

In [585]:
X_train = pd.read_csv("../data/X_train.csv", index_col=False)
X_train = X_train.loc[:, ~X_train.columns.str.contains('^Unnamed')]

X_val = pd.read_csv("../data/X_val.csv", index_col=False)
X_val = X_val.loc[:, ~X_val.columns.str.contains('^Unnamed')]

X_test = pd.read_csv("../data/X_test.csv", index_col=False)
X_test = X_test.loc[:, ~X_test.columns.str.contains('^Unnamed')]

y_train = pd.read_csv("../data/y_train.csv", index_col=False)
y_train = y_train.loc[:, ~y_train.columns.str.contains('^Unnamed')]

y_val = pd.read_csv("../data/y_val.csv", index_col=False)
y_val = y_val.loc[:, ~y_val.columns.str.contains('^Unnamed')]

y_test = pd.read_csv("../data/y_test.csv", index_col=False)
y_test = y_test.loc[:, ~y_test.columns.str.contains('^Unnamed')]

C:\Users\ntama\AppData\Local\Temp\ipykernel_2732\4027156877.py:7: DtypeWarning: Columns (8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40) have mixed types. Specify dtype option on import or set low_memory=False.
  X_test = pd.read_csv("../data/X_test.csv", index_col=False)


In [586]:
X_train = X_train.drop(['sex'], axis=1)
X_val = X_val.drop(['sex'], axis=1)
X_test = X_test.drop(['sex'], axis=1)

X_train = X_train.drop(['height'], axis=1)
X_val = X_val.drop(['height'], axis=1)
X_test = X_test.drop(['height'], axis=1)

X_train = X_train.drop(['age'], axis=1)
X_val = X_val.drop(['age'], axis=1)
X_test = X_test.drop(['age'], axis=1)

In [587]:
print(X_train.dtypes[X_train.dtypes == 'object'])  # Debe salir vacío
print(X_val.dtypes[X_val.dtypes == 'object'])      # También debe estar vacío
print(X_test.dtypes[X_test.dtypes == 'object'])    # Igual


Series([], dtype: object)
Series([], dtype: object)
diet_anything                        object
diet_halal                           object
diet_kosher                          object
diet_other                           object
diet_vegan                           object
diet_vegetarian                      object
drinks_a little                      object
drinks_a lot                         object
drinks_not at all                    object
drugs_never                          object
drugs_often                          object
drugs_sometimes                      object
smokes_drinks_no_a little            object
smokes_drinks_no_a lot               object
smokes_drinks_no_not at all          object
smokes_drinks_yes_a little           object
smokes_drinks_yes_a lot              object
smokes_drinks_yes_not at all         object
drinks_drugs_a little_never          object
drinks_drugs_a little_often          object
drinks_drugs_a little_sometimes      object
drinks_drugs_a lot_never

## Checking class imbalance

In [588]:
print(y_train.value_counts())

body_type
0            35519
1             6443
Name: count, dtype: int64


Classes 2 and 3 are minoritarian, let's address the issue with oversampling

### RandomOverSampler on train set for class balance

In [589]:
ros = RandomOverSampler(random_state=42)
X_train_res, y_train_res = ros.fit_resample(X_train, y_train)

## Model initialization

Calculating weights for each class 

In [590]:
# Asegúrate de que y_train sea una Series
if isinstance(y_train, pd.DataFrame):
    y_train = y_train.squeeze()

# Repite para y_val si es necesario
if isinstance(y_val, pd.DataFrame):
    y_val = y_val.squeeze()


# Calcular ratio para clases desbalanceadas
pos = np.sum(y_train == 1)
neg = np.sum(y_train == 0)
scale_pos_weight = neg / pos

# Definir el modelo con ajuste para desbalance
model = XGBClassifier(
    use_label_encoder=False,
    eval_metric='logloss',
    scale_pos_weight=scale_pos_weight,
    random_state=42,
    n_jobs=-1
)

# Entrenar el modelo con validación
model.fit(
    X_train,
    y_train,
    eval_set=[(X_val, y_val)],
    verbose=True
)

# Predecir en validación
y_val_pred = model.predict(X_val)

# Reporte de métricas
print(classification_report(y_val, y_val_pred))
print(confusion_matrix(y_val, y_val_pred))

[0]	validation_0-logloss:0.67784
[1]	validation_0-logloss:0.67006
[2]	validation_0-logloss:0.66583
[3]	validation_0-logloss:0.66347
[4]	validation_0-logloss:0.66203
[5]	validation_0-logloss:0.66097
[6]	validation_0-logloss:0.66060
[7]	validation_0-logloss:0.66000
[8]	validation_0-logloss:0.65982
[9]	validation_0-logloss:0.65966
[10]	validation_0-logloss:0.65958
[11]	validation_0-logloss:0.65915
[12]	validation_0-logloss:0.65902
[13]	validation_0-logloss:0.65906
[14]	validation_0-logloss:0.65883
[15]	validation_0-logloss:0.65852
[16]	validation_0-logloss:0.65859
[17]	validation_0-logloss:0.65872
[18]	validation_0-logloss:0.65837
[19]	validation_0-logloss:0.65836
[20]	validation_0-logloss:0.65802
[21]	validation_0-logloss:0.65785
[22]	validation_0-logloss:0.65775
[23]	validation_0-logloss:0.65746
[24]	validation_0-logloss:0.65746
[25]	validation_0-logloss:0.65744
[26]	validation_0-logloss:0.65744
[27]	validation_0-logloss:0.65710
[28]	validation_0-logloss:0.65698
[29]	validation_0-loglos

c:\Users\ntama\anaconda3\Lib\site-packages\xgboost\training.py:183: UserWarning: [18:51:47] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


[49]	validation_0-logloss:0.65598
[50]	validation_0-logloss:0.65614
[51]	validation_0-logloss:0.65616
[52]	validation_0-logloss:0.65629
[53]	validation_0-logloss:0.65635
[54]	validation_0-logloss:0.65639
[55]	validation_0-logloss:0.65606
[56]	validation_0-logloss:0.65623
[57]	validation_0-logloss:0.65617
[58]	validation_0-logloss:0.65582
[59]	validation_0-logloss:0.65576
[60]	validation_0-logloss:0.65572
[61]	validation_0-logloss:0.65582
[62]	validation_0-logloss:0.65571
[63]	validation_0-logloss:0.65544
[64]	validation_0-logloss:0.65541
[65]	validation_0-logloss:0.65545
[66]	validation_0-logloss:0.65550
[67]	validation_0-logloss:0.65539
[68]	validation_0-logloss:0.65536
[69]	validation_0-logloss:0.65564
[70]	validation_0-logloss:0.65541
[71]	validation_0-logloss:0.65551
[72]	validation_0-logloss:0.65537
[73]	validation_0-logloss:0.65559
[74]	validation_0-logloss:0.65559
[75]	validation_0-logloss:0.65558
[76]	validation_0-logloss:0.65539
[77]	validation_0-logloss:0.65522
[78]	validatio

### Training the model

Predictions

In [591]:
model.feature_importances_

array([0.02582346, 0.05810856, 0.00710198, 0.        , 0.02677118,
       0.01809508, 0.02154698, 0.02186665, 0.01237319, 0.03181646,
       0.02134009, 0.02577575, 0.02134413, 0.03248583, 0.02438657,
       0.02638083, 0.03587963, 0.02511702, 0.10463815, 0.01834345,
       0.02489654, 0.03023632, 0.0273184 , 0.01954128, 0.03203364,
       0.01572843, 0.0165448 , 0.02459431, 0.01086344, 0.02270394,
       0.02166272, 0.00335854, 0.01140266, 0.0533203 , 0.02309667,
       0.02690162, 0.03215736, 0.00937109, 0.03507294], dtype=float32)

In [592]:
xgb = XGBClassifier(
    objective='binary:logistic',
    use_label_encoder=False,
    scale_pos_weight=scale_pos_weight,
    n_jobs=-1,
    random_state=42,
    eval_metric='logloss'
)

param_grid = {
    'n_estimators': [100, 200],
    'learning_rate': [0.05, 0.1],
    'max_depth': [3, 5],
    'min_child_weight': [1, 3],
    'subsample': [0.8, 1.0],
    'colsample_bytree': [0.8, 1.0],
    'gamma': [0, 0.2]
}

grid_search = GridSearchCV(
    estimator=xgb,
    param_grid=param_grid,
    scoring='recall',  # ⚠️ Enfocado en sensibilidad de clase positiva
    cv=3,
)


grid_search.fit(X_train, y_train)

c:\Users\ntama\anaconda3\Lib\site-packages\xgboost\training.py:183: UserWarning: [18:51:47] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
c:\Users\ntama\anaconda3\Lib\site-packages\xgboost\training.py:183: UserWarning: [18:51:48] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
c:\Users\ntama\anaconda3\Lib\site-packages\xgboost\training.py:183: UserWarning: [18:51:48] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
c:\Users\ntama\anaconda3\Lib\site-packages\xgboost\training.py:183: UserWarning: [18:51:48] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtr

GridSearchCV(cv=3,
             estimator=XGBClassifier(base_score=None, booster=None,
                                     callbacks=None, colsample_bylevel=None,
                                     colsample_bynode=None,
                                     colsample_bytree=None, device=None,
                                     early_stopping_rounds=None,
                                     enable_categorical=False,
                                     eval_metric='logloss', feature_types=None,
                                     feature_weights=None, gamma=None,
                                     grow_policy=None, importance_type=None,
                                     interaction_constraint...
                                     max_delta_step=None, max_depth=None,
                                     max_leaves=None, min_child_weight=None,
                                     missing=nan, monotone_constraints=None,
                                     multi_strategy=None, n_estimators=None,
                                     n_jobs=-1, num_parallel_tree=None, ...),
             param_grid={'colsample_bytree': [0.8, 1.0], 'gamma': [0, 0.2],
                         'learning_rate': [0.05, 0.1], 'max_depth': [3, 5],
                         'min_child_weight': [1, 3], 'n_estimators': [100, 200],
                         'subsample': [0.8, 1.0]},
             scoring='recall')

In [593]:
best_model = grid_search.best_estimator_

y_val_pred = best_model.predict(X_val)

print("Mejores hiperparámetros:", grid_search.best_params_)
print("\nReporte de clasificación:")
print(classification_report(y_val, y_val_pred))
print("\nMatriz de confusión:")
print(confusion_matrix(y_val, y_val_pred))

Mejores hiperparámetros: {'colsample_bytree': 1.0, 'gamma': 0, 'learning_rate': 0.05, 'max_depth': 3, 'min_child_weight': 3, 'n_estimators': 100, 'subsample': 0.8}

Reporte de clasificación:
              precision    recall  f1-score   support

           0       0.90      0.56      0.69      7611
           1       0.21      0.64      0.31      1381

    accuracy                           0.57      8992
   macro avg       0.55      0.60      0.50      8992
weighted avg       0.79      0.57      0.63      8992


Matriz de confusión:
[[4235 3376]
 [ 495  886]]


In [594]:
from sklearn.metrics import precision_recall_fscore_support

y_proba = best_model.predict_proba(X_val)[:, 1]  # Probabilidad de clase positiva

thresholds = np.arange(0.1, 0.9, 0.05)
for thresh in thresholds:
    y_pred_thresh = (y_proba >= thresh).astype(int)
    precision, recall, f1, _ = precision_recall_fscore_support(y_val, y_pred_thresh, average='binary')
    print(f"Umbral: {thresh:.2f} | Precision: {precision:.3f} | Recall: {recall:.3f} | F1: {f1:.3f}")


Umbral: 0.10 | Precision: 0.154 | Recall: 1.000 | F1: 0.266
Umbral: 0.15 | Precision: 0.154 | Recall: 1.000 | F1: 0.266
Umbral: 0.20 | Precision: 0.154 | Recall: 1.000 | F1: 0.266
Umbral: 0.25 | Precision: 0.154 | Recall: 1.000 | F1: 0.266
Umbral: 0.30 | Precision: 0.161 | Recall: 0.970 | F1: 0.276
Umbral: 0.35 | Precision: 0.175 | Recall: 0.885 | F1: 0.293
Umbral: 0.40 | Precision: 0.185 | Recall: 0.827 | F1: 0.302
Umbral: 0.45 | Precision: 0.200 | Recall: 0.707 | F1: 0.312
Umbral: 0.50 | Precision: 0.208 | Recall: 0.642 | F1: 0.314
Umbral: 0.55 | Precision: 0.223 | Recall: 0.492 | F1: 0.307
Umbral: 0.60 | Precision: 0.246 | Recall: 0.318 | F1: 0.277
Umbral: 0.65 | Precision: 0.276 | Recall: 0.135 | F1: 0.182
Umbral: 0.70 | Precision: 0.344 | Recall: 0.046 | F1: 0.081
Umbral: 0.75 | Precision: 0.538 | Recall: 0.005 | F1: 0.010
Umbral: 0.80 | Precision: 0.000 | Recall: 0.000 | F1: 0.000
Umbral: 0.85 | Precision: 0.000 | Recall: 0.000 | F1: 0.000


c:\Users\ntama\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\ntama\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
